In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [2]:
import tensorflow as tf
import numpy as np

In [3]:
(X_data, y_data), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
print (X_data.shape)
# reduce the dimention of X_train, X_test
def reduce_D (data):
    return data.astype (np.float32).reshape (-1, 28 * 28) / 255.0
X_data, X_test = reduce_D  (X_data), reduce_D (X_test)
print (X_data.shape, X_test.shape)

# Check Y :
# print (y_data.shape)
# def y_preprocess (data):
#     return data.astype (np.int32).reshape (data.shape[0], 1)
# y_data, y_test = y_preprocess (y_data), y_preprocess (y_test)
print (y_data.shape, y_test.shape)

# split the train and validation data:
X_validation, y_validation = X_data[:5000], y_data[:5000]
X_train, y_train = X_data[5000:], y_data[5000:]

print (X_validation.shape, X_train.shape, y_validation.shape)

(60000, 28, 28)
(60000, 784) (10000, 784)
(60000,) (10000,)
(5000, 784) (55000, 784) (5000,)


## Regular GD

Step up: 5 layers with 100 nodes on each layer, mini-batch size is 32, train 20 steps.

Performance evaluation: 

    time consumption: 89.35848474502563
    performance score: 0.9733

In [4]:
N_feature = X_test.shape[1]
n_layers = 5
n_neurons = 100
learning_rate = 0.01
n_output = 10

In [5]:
X = tf.placeholder (tf.float32, shape = (None, N_feature), name = 'X')
y = tf.placeholder (tf.int32, shape = (None), name = 'y')

In [6]:
with tf.name_scope ('DNN_chapter11_8'):
    he_init = tf.contrib.layers.variance_scaling_initializer ()
    hidden1 = tf.layers.dense (X, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden1')
    hidden2 = tf.layers.dense (hidden1, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden2')
    hidden3 = tf.layers.dense (hidden2, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden3')
    hidden4 = tf.layers.dense (hidden3, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden4')
    hidden5 = tf.layers.dense (hidden4, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden5')
    y_predit = tf.layers.dense (hidden5, n_output, kernel_initializer = he_init, name = 'Output')
    y_proba = tf.nn.softmax (y_predit, name = "y_predit")

In [7]:
with tf.name_scope ('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_predit)
    loss = tf.reduce_mean(xentropy, name="loss")

In [8]:
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [9]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(y_predit, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [10]:
init = tf.global_variables_initializer()
n_steps = 20
batch_size = 32
data_size = X_train.shape[0]
n_batch = int (np.ceil (data_size / batch_size))

In [11]:
import time

In [12]:
with tf.Session() as sess:
    init.run()
    total_start_time = time.time()
    for step in range (n_steps):
        start_time = time.time ()
        for i_batch in range (n_batch):
            start = i_batch * batch_size
            if i_batch != n_batch - 1:
                end = start + batch_size
            else:
                end = data_size
            sess.run (training_op, feed_dict = {X: X_data[start: end], y: y_data[start: end]})
        train_score = accuracy.eval (feed_dict = {X: X_data[start: end], y: y_data[start: end]})
        validation_score = accuracy.eval (feed_dict = {X: X_validation, y:y_validation})
        end_time = time.time ()
        run_time = end_time - start_time
        print ('At step ', step, 'the train_score is ', train_score, 'the validation score is ', validation_score, 'this cost', run_time)
    total_end_time = time.time ()
    accuracy_val = accuracy.eval (feed_dict = {X: X_test, y: y_test})
    print ('Run', n_steps,  'steps total process time is',  total_end_time - total_start_time)
    print ('the test set accuracy is', accuracy_val)

At step  0 the train_score is  0.9166667 the validation score is  0.9386 this cost 4.622397422790527
At step  1 the train_score is  0.9166667 the validation score is  0.957 this cost 4.453064918518066
At step  2 the train_score is  1.0 the validation score is  0.9644 this cost 4.459587097167969
At step  3 the train_score is  1.0 the validation score is  0.9686 this cost 4.455009937286377
At step  4 the train_score is  1.0 the validation score is  0.972 this cost 4.459731817245483
At step  5 the train_score is  1.0 the validation score is  0.975 this cost 4.4487059116363525
At step  6 the train_score is  1.0 the validation score is  0.976 this cost 4.45017147064209
At step  7 the train_score is  1.0 the validation score is  0.9784 this cost 4.459004163742065
At step  8 the train_score is  1.0 the validation score is  0.9822 this cost 4.527098178863525
At step  9 the train_score is  1.0 the validation score is  0.9842 this cost 4.454243421554565
At step  10 the train_score is  1.0 the va

In [49]:
reset_graph ()
N_feature = X_test.shape[1]
n_layers = 5
n_neurons = 100
learning_rate = 0.001
n_output = 10

## Adam:
Step up: 5 layers with 100 nodes on each layer, mini-batch size is 32, train 30 steps.

Performance evaluation:

    time consumption:  114.90625596046448
    performance score:   0.1998

Appearently there is an error in decent, it seems moved too fast and missed the lowest point, we shall introduce the earily stop tech.
Idealy, at step 15 , you should stop the process. The train score dropped to  0.45833334 at step 15 same as validation score

Change the learning rate to 0.001 could resolve this issue, although I still don't understand why 

In [50]:
X = tf.placeholder (tf.float32, shape = (None, N_feature), name = 'X')
y = tf.placeholder (tf.int32, shape = (None), name = 'y')

In [51]:
with tf.name_scope ('DNN_chapter11_8'):
    he_init = tf.contrib.layers.variance_scaling_initializer ()
    hidden1 = tf.layers.dense (X, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden1')
    hidden2 = tf.layers.dense (hidden1, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden2')
    hidden3 = tf.layers.dense (hidden2, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden3')
    hidden4 = tf.layers.dense (hidden3, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden4')
    hidden5 = tf.layers.dense (hidden4, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden5')
    y_predit = tf.layers.dense (hidden5, n_output, kernel_initializer = he_init, name = 'Output')

In [52]:
with tf.name_scope ('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_predit)
    loss = tf.reduce_mean(xentropy, name="loss")

In [53]:
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [54]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(y_predit, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [55]:
init = tf.global_variables_initializer()
n_steps = 20
batch_size = 32
data_size = X_train.shape[0]
n_batch = int (np.ceil (data_size / batch_size))

In [56]:
with tf.Session() as sess:
    init.run()
    total_start_time = time.time()
    for step in range (n_steps):
        start_time = time.time ()
        for i_batch in range (n_batch):
            start = i_batch * batch_size
            if i_batch != n_batch - 1:
                end = start + batch_size
            else:
                end = data_size
            sess.run (training_op, feed_dict = {X: X_train[start: end], y: y_train[start: end]})
        loss_train, train_score = sess.run ([loss, accuracy], feed_dict = {X: X_train[start: end], y: y_train[start: end]})
        loss_val, validation_score = sess.run([loss, accuracy], feed_dict = {X: X_validation, y:y_validation})
        end_time = time.time ()
        run_time = end_time - start_time
        print ('At step ', step, 'the train_score is ', train_score, 'the validation score is ', validation_score, 'this cost', run_time)
        print ('At step ', step, 'the loss is ', loss_val, 'the best loss is ', best_loss, 'this cost', run_time)
        print (loss_train)
    total_end_time = time.time ()
    accuracy_val = accuracy.eval (feed_dict = {X: X_test, y: y_test})
    print ('Run', n_steps,  'steps total process time is',  total_end_time - total_start_time)
    print ('the test set accuracy is', accuracy_val)

At step  0 the train_score is  1.0 the validation score is  0.9478 this cost 6.27926230430603
At step  0 the loss is  0.17681785 the best loss is  1000 this cost 6.27926230430603
0.023942286
At step  1 the train_score is  1.0 the validation score is  0.963 this cost 6.287681818008423
At step  1 the loss is  0.13302934 the best loss is  1000 this cost 6.287681818008423
0.015892332
At step  2 the train_score is  1.0 the validation score is  0.9698 this cost 6.267089128494263
At step  2 the loss is  0.11656096 the best loss is  1000 this cost 6.267089128494263
0.0004148536
At step  3 the train_score is  1.0 the validation score is  0.9704 this cost 6.497232913970947
At step  3 the loss is  0.1071388 the best loss is  1000 this cost 6.497232913970947
0.0028896108
At step  4 the train_score is  1.0 the validation score is  0.9628 this cost 6.320879697799683
At step  4 the loss is  0.13623112 the best loss is  1000 this cost 6.320879697799683
0.00039440638
At step  5 the train_score is  1.0 

## Adam with early stop:

Step up: 5 layers with 100 nodes on each layer, mini-batch size is 32, train 20 steps. earily stop if the loss not been reduced further for 35% of the steps.(at step 2, it seems randomly find a good result so stop earily)

Performance evaluation:

    time consumption: 51.90262746810913
    performance score: 0.9
    
Step up: 5 layers with 100 nodes on each layer, mini-batch size is 32, train 20 steps. earily stop if the loss not been reduced further for 30% of the steps.

Performance evaluation:

    time consumption: 61.913670778274536
    performance score: 0.8656



In [21]:
reset_graph ()

In [22]:
X = tf.placeholder (tf.float32, shape = (None, N_feature), name = 'X')
y = tf.placeholder (tf.int32, shape = (None), name = 'y')

In [23]:
with tf.name_scope ('DNN_chapter11_8'):
    he_init = tf.contrib.layers.variance_scaling_initializer ()
    hidden1 = tf.layers.dense (X, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden1')
    hidden2 = tf.layers.dense (hidden1, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden2')
    hidden3 = tf.layers.dense (hidden2, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden3')
    hidden4 = tf.layers.dense (hidden3, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden4')
    hidden5 = tf.layers.dense (hidden4, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden5')
    y_predit = tf.layers.dense (hidden5, n_output, kernel_initializer = he_init, name = 'Output')

In [24]:
with tf.name_scope ('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_predit)
    loss = tf.reduce_mean(xentropy, name="loss")

In [25]:
with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [26]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(y_predit, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [27]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()
n_steps = 20
batch_size = 32
data_size = X_train.shape[0]
n_batch = int (np.ceil (data_size / batch_size))

In [28]:
with tf.Session() as sess:
    init.run()
    total_start_time = time.time()
    best_loss = 1000 
    without_update_counter = 0
    max_without_update = int (0.35 * n_steps)
    for step in range (n_steps):
        start_time = time.time ()
        for i_batch in range (n_batch):
            start = i_batch * batch_size
            if i_batch != n_batch - 1:
                end = start + batch_size
            else:
                end = data_size
            sess.run (training_op, feed_dict = {X: X_train[start: end], y: y_train[start: end]})
        train_score = accuracy.eval (feed_dict = {X: X_train[start: end], y: y_train[start: end]})
        loss_val, validation_score = sess.run([loss, accuracy], feed_dict = {X: X_validation, y:y_validation})
        end_time = time.time ()
        run_time = end_time - start_time
        print ('At step ', step, 'the train_score is ', train_score, 'the validation score is ', validation_score, 'this cost', run_time)
        print ('At step ', step, 'the loss is ', loss_val, 'the best loss is ', best_loss, 'this cost', run_time)
        if loss_val < best_loss:
            save_path = saver.save (sess, r'C:\Users\asiaynrf\Project\model_saver\my_model.ckpt')
            best_loss = loss_val
            without_update_counter = 0
        else:
            without_update_counter += 1
            if without_update_counter > max_without_update:
                print ('earily stopped')
                break
    total_end_time = time.time ()
    print ('Run', n_steps,  'steps total process time is',  total_end_time - total_start_time)

At step  0 the train_score is  1.0 the validation score is  0.9096 this cost 5.754924535751343
At step  0 the loss is  0.39634955 the best loss is  1000 this cost 5.754924535751343
At step  1 the train_score is  0.875 the validation score is  0.7856 this cost 5.699921131134033
At step  1 the loss is  1.0206443 the best loss is  0.39634955 this cost 5.699921131134033
At step  2 the train_score is  0.625 the validation score is  0.6014 this cost 5.742764234542847
At step  2 the loss is  1.4077932 the best loss is  0.39634955 this cost 5.742764234542847
At step  3 the train_score is  1.0 the validation score is  0.8964 this cost 5.714359283447266
At step  3 the loss is  0.527578 the best loss is  0.39634955 this cost 5.714359283447266
At step  4 the train_score is  0.8333333 the validation score is  0.7586 this cost 5.752542495727539
At step  4 the loss is  0.8203664 the best loss is  0.39634955 this cost 5.752542495727539
At step  5 the train_score is  0.875 the validation score is  0.82

In [29]:
with tf.Session() as sess:
    saver.restore (sess, r'C:\Users\asiaynrf\Project\model_saver\my_model.ckpt')
    accuracy_val = accuracy.eval (feed_dict = {X: X_test, y: y_test})
    print ('the test set accuracy is', accuracy_val)

INFO:tensorflow:Restoring parameters from C:\Users\asiaynrf\Project\model_saver\my_model.ckpt
the test set accuracy is 0.9


## GD without mini-batch
Performance evaluation: 

Score: (fogot to run the test set) 2000 steps's validation score is  0.9448. For the previous setup, it is 32 batch size, in another word, one step is an 1719 steps, so 20 step is 34380 steps in total. No wonder why mini-batch is better.

Take look at the time consumption, it is 1468s, compare to 120s in previous method to have similar (actually worse) performance. 

time consume: 1483.5600037574768

In [30]:
reset_graph()

In [31]:
X = tf.placeholder (tf.float32, shape = (None, N_feature), name = 'X')
y = tf.placeholder (tf.int32, shape = (None), name = 'y')

In [32]:
with tf.name_scope ('DNN_chapter11_8'):
    he_init = tf.contrib.layers.variance_scaling_initializer ()
    hidden1 = tf.layers.dense (X, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden1')
    hidden2 = tf.layers.dense (hidden1, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden2')
    hidden3 = tf.layers.dense (hidden2, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden3')
    hidden4 = tf.layers.dense (hidden3, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden4')
    hidden5 = tf.layers.dense (hidden4, n_neurons, activation = tf.nn.elu, kernel_initializer = he_init, name = 'Hidden5')
    y_predit = tf.layers.dense (hidden5, n_output, kernel_initializer = he_init, name = 'Output')

In [33]:
with tf.name_scope ('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=y_predit)
    loss = tf.reduce_mean(xentropy, name="loss")

In [34]:
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [35]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(y_predit, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [36]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()
n_steps = 2000
batch_size = 32
data_size = X_train.shape[0]
n_batch = int (np.ceil (data_size / batch_size))

In [37]:
with tf.Session() as sess:
    init.run()
    total_start_time = time.time()
    best_loss = 1000 
    without_update_counter = 0
    max_without_update = int (0.2 * n_steps)
    for step in range (n_steps):
        start_time = time.time ()
        sess.run (training_op, feed_dict = {X: X_train, y: y_train})
        train_score = accuracy.eval (feed_dict = {X: X_train, y: y_train})
        loss_val, validation_score = sess.run([loss, accuracy], feed_dict = {X: X_validation, y:y_validation})
        end_time = time.time ()
        run_time = end_time - start_time
        print ('At step ', step, 'the train_score is ', train_score, 'the validation score is ', validation_score, 'this cost', run_time)
        # print ('At step ', step, 'the loss is ', loss_val, 'the best loss is ', best_loss, 'this cost', run_time)
#         if loss_val < best_loss:
#             save_path = saver.save (sess, r'C:\Users\asiaynrf\Project\model_saver\my_model.ckpt')
#             best_loss = loss_val
#             without_update_counter = 0
#         else:
#             without_update_counter += 1
#             if without_update_counter > max_without_update:
#                 print ('earily stopped')
#                 break
    total_end_time = time.time ()
    print ('Run', n_steps,  'steps total process time is',  total_end_time - total_start_time)

At step  0 the train_score is  0.08370909 the validation score is  0.0824 this cost 0.8053779602050781
At step  1 the train_score is  0.09447273 the validation score is  0.092 this cost 0.7447667121887207
At step  2 the train_score is  0.110581815 the validation score is  0.1078 this cost 0.7412323951721191
At step  3 the train_score is  0.1326 the validation score is  0.1272 this cost 0.7435576915740967
At step  4 the train_score is  0.15823637 the validation score is  0.1538 this cost 0.7295303344726562
At step  5 the train_score is  0.18592727 the validation score is  0.1806 this cost 0.7557237148284912
At step  6 the train_score is  0.21798182 the validation score is  0.2136 this cost 0.742232084274292
At step  7 the train_score is  0.25236362 the validation score is  0.2492 this cost 0.7508418560028076
At step  8 the train_score is  0.2888 the validation score is  0.2824 this cost 0.7431344985961914
At step  9 the train_score is  0.32336363 the validation score is  0.3176 this cos

At step  80 the train_score is  0.7868 the validation score is  0.794 this cost 0.748999834060669
At step  81 the train_score is  0.78845453 the validation score is  0.7946 this cost 0.7439157962799072
At step  82 the train_score is  0.7900364 the validation score is  0.7972 this cost 0.7389805316925049
At step  83 the train_score is  0.7916545 the validation score is  0.7984 this cost 0.7439911365509033
At step  84 the train_score is  0.7930545 the validation score is  0.8008 this cost 0.7405147552490234
At step  85 the train_score is  0.7943818 the validation score is  0.803 this cost 0.7434859275817871
At step  86 the train_score is  0.7956909 the validation score is  0.804 this cost 0.742168664932251
At step  87 the train_score is  0.7968909 the validation score is  0.8046 this cost 0.7443511486053467
At step  88 the train_score is  0.7979818 the validation score is  0.8052 this cost 0.7417969703674316
At step  89 the train_score is  0.79914546 the validation score is  0.806 this c

At step  160 the train_score is  0.8502 the validation score is  0.8562 this cost 0.7411162853240967
At step  161 the train_score is  0.85056365 the validation score is  0.8566 this cost 0.7415461540222168
At step  162 the train_score is  0.85121816 the validation score is  0.8572 this cost 0.7422316074371338
At step  163 the train_score is  0.85156363 the validation score is  0.8574 this cost 0.7402100563049316
At step  164 the train_score is  0.8520182 the validation score is  0.858 this cost 0.7407631874084473
At step  165 the train_score is  0.8524 the validation score is  0.858 this cost 0.7397301197052002
At step  166 the train_score is  0.85261816 the validation score is  0.8586 this cost 0.7399852275848389
At step  167 the train_score is  0.853 the validation score is  0.8596 this cost 0.7434847354888916
At step  168 the train_score is  0.85345453 the validation score is  0.8602 this cost 0.7400658130645752
At step  169 the train_score is  0.8537273 the validation score is  0.8

At step  240 the train_score is  0.8730909 the validation score is  0.8814 this cost 0.7490251064300537
At step  241 the train_score is  0.87332726 the validation score is  0.8814 this cost 0.7395401000976562
At step  242 the train_score is  0.8735273 the validation score is  0.8814 this cost 0.7488269805908203
At step  243 the train_score is  0.87372726 the validation score is  0.8818 this cost 0.7399873733520508
At step  244 the train_score is  0.8739273 the validation score is  0.8818 this cost 0.7391505241394043
At step  245 the train_score is  0.87407273 the validation score is  0.8818 this cost 0.741157054901123
At step  246 the train_score is  0.87430906 the validation score is  0.8822 this cost 0.7407991886138916
At step  247 the train_score is  0.8745091 the validation score is  0.8826 this cost 0.7413475513458252
At step  248 the train_score is  0.87470907 the validation score is  0.8828 this cost 0.7393717765808105
At step  249 the train_score is  0.8748909 the validation sc

At step  319 the train_score is  0.8862727 the validation score is  0.8916 this cost 0.7401731014251709
At step  320 the train_score is  0.8864727 the validation score is  0.8918 this cost 0.7426035404205322
At step  321 the train_score is  0.8866182 the validation score is  0.8916 this cost 0.7413594722747803
At step  322 the train_score is  0.8868 the validation score is  0.8916 this cost 0.7384710311889648
At step  323 the train_score is  0.88690907 the validation score is  0.8918 this cost 0.7401907444000244
At step  324 the train_score is  0.88705456 the validation score is  0.8918 this cost 0.7384076118469238
At step  325 the train_score is  0.8872 the validation score is  0.8918 this cost 0.7390029430389404
At step  326 the train_score is  0.8873091 the validation score is  0.8918 this cost 0.7427048683166504
At step  327 the train_score is  0.88745457 the validation score is  0.8918 this cost 0.7417736053466797
At step  328 the train_score is  0.8875818 the validation score is 

...

At step  1662 the train_score is  0.93945456 the validation score is  0.9408 this cost 0.7489314079284668
At step  1663 the train_score is  0.93947273 the validation score is  0.9408 this cost 0.7396905422210693
At step  1664 the train_score is  0.9394909 the validation score is  0.9408 this cost 0.7375171184539795
At step  1665 the train_score is  0.9395273 the validation score is  0.9408 this cost 0.7399327754974365
At step  1666 the train_score is  0.93954545 the validation score is  0.9408 this cost 0.7469327449798584
At step  1667 the train_score is  0.93956363 the validation score is  0.9412 this cost 0.7410116195678711
At step  1668 the train_score is  0.93956363 the validation score is  0.9412 this cost 0.7426192760467529
At step  1669 the train_score is  0.9396 the validation score is  0.9412 this cost 0.7379581928253174
At step  1670 the train_score is  0.93963635 the validation score is  0.9414 this cost 0.7397801876068115
At step  1671 the train_score is  0.93963635 the val

At step  1741 the train_score is  0.94085455 the validation score is  0.943 this cost 0.7393443584442139
At step  1742 the train_score is  0.9408727 the validation score is  0.943 this cost 0.7407963275909424
At step  1743 the train_score is  0.9409091 the validation score is  0.943 this cost 0.7355420589447021
At step  1744 the train_score is  0.9409091 the validation score is  0.943 this cost 0.7456023693084717
At step  1745 the train_score is  0.94092727 the validation score is  0.943 this cost 0.7391374111175537
At step  1746 the train_score is  0.9409636 the validation score is  0.943 this cost 0.740464448928833
At step  1747 the train_score is  0.9409636 the validation score is  0.943 this cost 0.7400331497192383
At step  1748 the train_score is  0.941 the validation score is  0.943 this cost 0.7411797046661377
At step  1749 the train_score is  0.94101816 the validation score is  0.943 this cost 0.7418661117553711
At step  1750 the train_score is  0.9410545 the validation score i

At step  1820 the train_score is  0.9421818 the validation score is  0.9434 this cost 0.7403557300567627
At step  1821 the train_score is  0.9422182 the validation score is  0.9434 this cost 0.7415454387664795
At step  1822 the train_score is  0.9422182 the validation score is  0.9434 this cost 0.7403738498687744
At step  1823 the train_score is  0.9422182 the validation score is  0.9434 this cost 0.7413616180419922
At step  1824 the train_score is  0.94223636 the validation score is  0.9434 this cost 0.7405846118927002
At step  1825 the train_score is  0.94223636 the validation score is  0.9436 this cost 0.7385833263397217
At step  1826 the train_score is  0.94223636 the validation score is  0.9436 this cost 0.7413074970245361
At step  1827 the train_score is  0.9422182 the validation score is  0.9436 this cost 0.7401752471923828
At step  1828 the train_score is  0.9422182 the validation score is  0.9436 this cost 0.7404406070709229
At step  1829 the train_score is  0.94225454 the val

At step  1898 the train_score is  0.94307274 the validation score is  0.9442 this cost 0.7424166202545166
At step  1899 the train_score is  0.9430909 the validation score is  0.9442 this cost 0.7467176914215088
At step  1900 the train_score is  0.9431273 the validation score is  0.9442 this cost 0.7422780990600586
At step  1901 the train_score is  0.94314545 the validation score is  0.9442 this cost 0.7404239177703857
At step  1902 the train_score is  0.9432 the validation score is  0.9442 this cost 0.7429676055908203
At step  1903 the train_score is  0.9432182 the validation score is  0.9442 this cost 0.7413551807403564
At step  1904 the train_score is  0.94323635 the validation score is  0.9442 this cost 0.7405893802642822
At step  1905 the train_score is  0.9432909 the validation score is  0.9442 this cost 0.7414605617523193
At step  1906 the train_score is  0.94330907 the validation score is  0.9442 this cost 0.7407851219177246
At step  1907 the train_score is  0.94330907 the valid

At step  1976 the train_score is  0.9441818 the validation score is  0.9444 this cost 0.7428357601165771
At step  1977 the train_score is  0.9441818 the validation score is  0.9446 this cost 0.7400033473968506
At step  1978 the train_score is  0.9441818 the validation score is  0.9446 this cost 0.7438642978668213
At step  1979 the train_score is  0.9442 the validation score is  0.9446 this cost 0.742185115814209
At step  1980 the train_score is  0.94423634 the validation score is  0.9446 this cost 0.7401189804077148
At step  1981 the train_score is  0.94421816 the validation score is  0.9446 this cost 0.7477507591247559
At step  1982 the train_score is  0.94423634 the validation score is  0.9448 this cost 0.7461452484130859
At step  1983 the train_score is  0.9442545 the validation score is  0.9448 this cost 0.7427825927734375
At step  1984 the train_score is  0.9442545 the validation score is  0.9448 this cost 0.7396097183227539
At step  1985 the train_score is  0.94429094 the validat

In [38]:
y_train.shape

(55000,)

In [57]:
from tensorflow_graph_in_jupyter import show_graph
show_graph(tf.get_default_graph())